In [20]:
import pandas as pd
import numpy as np
import spacy
import re
from spacy.tokenizer import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
import torch
#from torchtext.data.utils import get_tokenizer
#from torchtext.vocab import build_vocab_from_iterator
import tensorflow as tf
from datasets import load_dataset

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [49]:
from transformers import BertModel, BertConfig,BertTokenizer, BertForSequenceClassification,AdamW, get_linear_schedule_with_warmup
from transformers.models.bert.modeling_bert import BertForPreTrainingOutput
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [3]:
configuration = BertConfig()
print(configuration)
model = BertModel(configuration)
print(model)
config = model.config    
print(config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            

In [5]:

#dataset = load_dataset(
#   'imdb')


#dataset['train'].to_csv('imdb_train.csv')
#dataset['test'].to_csv('imdb_test.csv')

Reusing dataset imdb (C:\Users\Aris\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

Creating CSV from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

32686819

In [4]:
dataset['train'][0]

NameError: name 'dataset' is not defined

In [4]:
df_train = pd.read_csv('imdb_train.csv')
df_test = pd.read_csv('imdb_test.csv')

print(df_train.head(10))
print(df_test.head(10))

   Unnamed: 0                                               text  label
0           0  I rented I AM CURIOUS-YELLOW from my video sto...      0
1           1  "I Am Curious: Yellow" is a risible and preten...      0
2           2  If only to avoid making this type of film in t...      0
3           3  This film was probably inspired by Godard's Ma...      0
4           4  Oh, brother...after hearing about this ridicul...      0
5           5  I would put this at the top of my list of film...      0
6           6  Whoever wrote the screenplay for this movie ob...      0
7           7  When I first saw a glimpse of this movie, I qu...      0
8           8  Who are these "They"- the actors? the filmmake...      0
9           9  This is said to be a personal film for Peter B...      0
   Unnamed: 0                                               text  label
0           0  I love sci-fi and am willing to put up with a ...      0
1           1  Worth the entertainment value of a rental, esp...

In [5]:
df_train['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [6]:
df_test['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [7]:
df_train

,Unnamed: 0,text,label
0,0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,2,If only to avoid making this type of film in t...,0
3,3,This film was probably inspired by Godard's Ma...,0
4,4,"Oh, brother...after hearing about this ridicul...",0
5,5,I would put this at the top of my list of film...,0
6,6,Whoever wrote the screenplay for this movie ob...,0
7,7,"When I first saw a glimpse of this movie, I qu...",0
8,8,"Who are these ""They""- the actors? the filmmake...",0
9,9,This is said to be a personal film for Peter B...,0


In [8]:
df_train['text'].to_csv("df_train.txt",index=False,header=True)
df_test['text'].to_csv("df_test.txt",index=False,header=True)


In [11]:
train_input = pd.read_csv("df_train.txt",delimiter="\t")
print("Training Data.....")
print(len(train_input))
print(train_input.iloc[0,:])

test_input = pd.read_csv("df_test.txt",delimiter="\t")
print("Test Data.....")
print(len(test_input))
print(test_input.iloc[0,:])

Training Data.....
25000
text    I rented I AM CURIOUS-YELLOW from my video sto...
Name: 0, dtype: object
Test Data.....
25000
text    I love sci-fi and am willing to put up with a ...
Name: 0, dtype: object


In [12]:
vocab = BertForPreTrainingOutput()
print(vocab)
token = BertTokenizer("df_train.txt")
print(token)

BertForPreTrainingOutput(loss=None, prediction_logits=None, seq_relationship_logits=None, hidden_states=None, attentions=None)
PreTrainedTokenizer(name_or_path='', vocab_size=24905, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [13]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [35]:
def preprocessing_data(df_input): 
    
    Ids = []
    Masks = []

    for i in range(len(df_input)):
        
        inputs = tokenizer.encode_plus(str(df_input.iloc[i,:]),add_special_tokens=True,
                padding='max_length',
                return_attention_mask=True,
                return_tensors="pt")
        Ids.append(inputs.get('input_ids'))
        Masks.append(inputs.get('attention_mask'))
    
    Ids = torch.cat(Ids)
    Masks = torch.cat(Masks)
    return Ids, Masks

In [36]:
## Get the Input Ids and Attention Masks for train and test data.

train_ids, train_masks = preprocessing_data(train_input)
test_ids, test_masks = preprocessing_data(test_input)

In [37]:
train_label = torch.tensor(df_train['label'])
test_label  = torch.tensor(df_test['label'])


In [39]:
## Create Train and Test dataset using DataLoader. 

batch_size = 32

train_data = TensorDataset(train_ids, train_masks, train_label)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)


test_data = TensorDataset(test_ids, test_masks, test_label)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
# Get the GPU device name.
device = torch.device("cuda")

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


for step,batch in enumerate(train_dataloader):
    
    input_ids = batch[0].to(device)    
    atten_masks = batch[1].to(device)
    labels = batch[2].to(device)
    
    model = BertForSequenceClassification.from_pretrained(
            "bert-base-uncased", 
            num_labels = 2,                     
            output_attentions = False, 
            output_hidden_states = False, 
                )
    
    optimizer = AdamW(model.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )
    model.zero_grad()
    
    logits = model(input_ids,atten_masks)
    
    print(logits)

No GPU available, using the CPU instead.


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at